In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor, HubertForCTC

# Load the pretrained HUBERT model and the corresponding processor
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft")

# Function to transcribe audio using the Hubert model
def transcribe_audio(file_path):
    # Load audio file
    waveform, sample_rate = torchaudio.load(file_path)
    
    # Ensure the audio file is in the correct format
    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
    
    # Ensure the waveform is 1D (single channel)
    if waveform.ndim > 1:
        waveform = waveform.mean(dim=0)  # Convert to mono by averaging channels

    # Preprocess the audio file
    inputs = processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
    
    # Perform inference
    with torch.no_grad():
        logits = model(**inputs).logits
    
    # Decode the output
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    
    return transcription[0]

# Example usage
audio_file = "corpus/clips/common_voice_pl_20547774.mp3"
transcription = transcribe_audio(audio_file)
print("Transcription:", transcription)

In [ ]:
from datasets import load_dataset

cv = load_dataset("mozilla-foundation/common_voice_17_0", "pl", split="train")
print(cv[0])

In [ ]:
import IPython

sample = cv[1]  # Change index as needed

print(f"Sentence: {sample['sentence']}")

audio_file = sample['path']
transcription = transcribe_audio(audio_file)
print("Transcription:", transcription)
IPython.display.Audio(data=audio_file, rate=16000)